In [1]:
! pip install cohere pinecone-client

In [2]:
cohere_api_key = ''
pinecone_api_key = ''

In [3]:
import cohere
co = cohere.Client(cohere_api_key)

In [4]:
from pinecone import Pinecone

pc = Pinecone(pinecone_api_key) # add your pinecone API key here

index_name = 'quranic'

# if the index does not exist, we create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=shape[1],
        metric='cosine'
    )

# connect to index
index = pc.Index(index_name)

In [5]:
limit = 1600

def retrieve(query):
    xq = co.embed(
        texts=[query],
        model='multilingual-22-12',
        truncate='NONE'
    ).embeddings
    # search pinecone index for context passage with the answer
    xc = index.query(vector=xq, top_k=3, include_metadata=True)
    contexts = [
        str(x['metadata']['Surah']) for x in xc['matches']
    ]
    contexts += [
        str(x['metadata']['Ayat']) for x in xc['matches']
    ]
    contexts += [
        str(x['metadata']['Arabic']) for x in xc['matches']
    ]
    contexts += [
        str(x['metadata']['Tafaseer1']) for x in xc['matches']
    ]
    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the Query based on the contexts, if it's not in the contexts say 'I don't know the answer'. \n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuery: {query}\nAnswer in the language of Query, if Query is in English Answer in English."
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt

In [6]:
query = "What does quran say about believers?"
query_with_contexts = retrieve(query)
query_with_contexts

"Answer the Query based on the contexts, if it's not in the contexts say 'I don't know the answer'. \n\nContext:\n8.0\n\n---\n\n2.0\n\n---\n\n24.0\n\n---\n\n2.0\n\n---\n\n8.0\n\n---\n\n47.0\n\n---\n\nإِنَّمَا ٱلْمُؤْمِنُونَ ٱلَّذِينَ إِذَا ذُكِرَ ٱللَّهُ وَجِلَتْ قُلُوبُهُمْ وَإِذَا تُلِيَتْ عَلَيْهِمْ ءَايَٰتُهُ زَادَتْهُمْ إِيمَٰناً وَعَلَىٰ رَبِّهِمْ يَتَوَكَّلُونَ\n\n---\n\nوَمِنَ ٱلنَّاسِ مَن يَقُولُ آمَنَّا بِٱللَّهِ وَبِٱلْيَوْمِ ٱلآخِرِ وَمَا هُم بِمُؤْمِنِينَ\n\n---\n\nوَيَقُولُونَ آمَنَّا بِٱللَّهِ وَبِٱلرَّسُولِ وَأَطَعْنَا ثُمَّ يَتَوَلَّىٰ فَرِيقٌ مِّنْهُمْ مِّن بَعْدِ ذٰلِكَ وَمَآ أُوْلَـٰئِكَ بِٱلْمُؤْمِنِينَ\n\n---\n\nThe believers those whose faith is complete are only those who when God is mentioned that is when His threat of punishment is mentioned their hearts tremble fear and when His verses are recited to them they increase their faith their acceptance of the truth and who rely upon their Lord who put their trust in Him alone and not in any other.\n\n---\n\nThe fo

In [8]:
def complete(prompt):
  response = co.generate(
                          model='c4ai-aya',
                          prompt=prompt,
                          max_tokens=300,
                          temperature=0.4,
                          k=0,
                          stop_sequences=[],
                          return_likelihoods='NONE'
                        )
  return response.generations[0].text

In [9]:
complete(query_with_contexts)

'The Quran says that believers are those who have faith in God and the Last Day. They are the ones who, when God is mentioned, their hearts tremble with fear and when His verses are recited to them, they increase their faith. They are the ones who put their trust in their Lord and not in any other.\n\nThe Quran also says that there are some people who say "We believe in God and the Last Day" but they are not believers. They are the hypocrites who say they believe in God and the Last Day but they do not believe in their hearts.\n\nThe Quran also says that believers are those who believe in God and the Last Day, and who obey God and the Messenger. They are the ones who, when God and His Messenger command them to do something, they do it. They are the ones who put their trust in God and the Messenger, and not in any other.'